In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt

def load_data():
    # Descriptive dataset
    data = {
        'user_id': [1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 8, 8, 9, 9, 10, 10],
        'age': [20, 23, 25, 26, 29, 30, 31, 33, 37, 20, 21, 25, 26, 27, 30, 31, 34, 35, 20, 25],
        'gender': ['Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
                   'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Female', 'Male', 'Female'],
        'genre': ['HipHop', 'HipHop', 'HipHop', 'Jazz', 'Jazz', 'Jazz', 'Classical', 'Classical', 'Classical',
                  'Dance', 'Dance', 'Dance', 'Acoustic', 'Acoustic', 'Acoustic', 'Classical', 'Classical', 'Classical', 'Jazz', 'Dance'],
        'rating': [5, 4, 3, 4, 5, 3, 5, 4, 3, 4, 5, 2, 3, 5, 4, 3, 5, 4, 5, 2]
    }
    return pd.DataFrame(data)

def add_user(data, user_id, age, gender, genre, rating):
    new_entry = pd.DataFrame({
        'user_id': [user_id],
        'age': [age],
        'gender': [gender],
        'genre': [genre],
        'rating': [rating]
    })
    return pd.concat([data, new_entry], ignore_index=True)

def create_user_item_matrix(data):
    # Use pivot_table to handle duplicates by averaging ratings
    return data.pivot_table(index='user_id', columns='genre', values='rating', aggfunc='mean').fillna(0)

def calculate_similarity(matrix, type='user'):
    if type == 'user':
        return cosine_similarity(matrix)
    elif type == 'item':
        return cosine_similarity(matrix.T)

def predict_ratings(similarity, user_item_matrix, type='user'):
    if type == 'user':
        mean_user_rating = user_item_matrix.mean(axis=1).values.reshape(-1, 1)
        ratings_diff = user_item_matrix - mean_user_rating
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        return pred
    elif type == 'item':
        return user_item_matrix.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

def recommend(predictions, user_id, user_item_matrix, n=5):
    user_index = user_id - 1  # Adjust for zero indexing
    sorted_indices = predictions[user_index].argsort()[::-1]
    unseen_items = np.where(user_item_matrix.iloc[user_index] == 0)[0]
    recommendations = [idx for idx in sorted_indices if idx in unseen_items][:n]
    return user_item_matrix.columns[recommendations].tolist()

def evaluate(predictions, test_matrix):
    mask = test_matrix > 0  # Only consider non-zero entries
    mse = mean_squared_error(test_matrix[mask], predictions[mask])
    return sqrt(mse)

# Menu interaction
def main():
    data = load_data()
    while True:
        print("\nMusic Recommendation System Menu:")
        print("1. View Dataset")
        print("2. Add User Rating")
        print("3. Generate Recommendations")
        print("4. Exit")

        choice = input("\nEnter your choice (1-4): ")

        if choice == '1':
            print("\nCurrent Dataset:")
            print(data)

        elif choice == '2':
            print("\nAdd a new user rating:")
            user_id = int(input("Enter User ID: "))
            age = int(input("Enter Age: "))
            gender = input("Enter Gender (Male/Female): ")
            genre = input("Enter Genre (HipHop/Jazz/Classical/Dance/Acoustic): ")
            rating = int(input("Enter Rating (1-5): "))

            data = add_user(data, user_id, age, gender, genre, rating)
            print("\nRating added successfully!")

        elif choice == '3':
            print("\nGenerating Recommendations...")
            user_item_matrix = create_user_item_matrix(data)
            similarity = calculate_similarity(user_item_matrix, type='user')
            predictions = predict_ratings(similarity, user_item_matrix, type='user')

            user_id = int(input("Enter User ID to get recommendations: "))
            recommendations = recommend(predictions, user_id, user_item_matrix, n=5)
            print(f"\nRecommended Genres for User {user_id}: {recommendations}")

        elif choice == '4':
            print("\nExiting the program. Goodbye!")
            break

        else:
            print("\nInvalid choice. Please try again.")

if __name__ == "__main__":
    main()



Music Recommendation System Menu:
1. View Dataset
2. Add User Rating
3. Generate Recommendations
4. Exit
